In [2]:
import pandas as pd
import numpy as np

from CNN_Audio_Model import CNN_Audio_Model
from TestDataset import TestDataset

import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl

import torchaudio
import os
import matplotlib.pyplot as plt
from pydub import AudioSegment

In [3]:
files_list = []
with open('wrong_classified_validation_file_names_25e.txt', 'r') as filehandle:
    for line in filehandle:
        # remove linebreak which is the last character of the string
        # currentPlace = line[:-1]
        #bez '.wav'
        currentPlace = line[:-1]+'.wav'
        # add item to the list
        files_list.append(currentPlace)

In [4]:

class Classifier():
    def __init__(self,path, file_name):
        self.path = path
        self.file = file_name
        self.model = CNN_Audio_Model.load_from_checkpoint(checkpoint_path="./model_25e_vallist.ckpt")
        self.trainer = pl.Trainer()
        #wczytanie wyniku z csv:
        self.csv_path = "./BirdVox/BirdVoxDCASE20k.csv"
        self.csv_data = pd.read_csv(self.csv_path)
        self.model_prediction = self.run_model()

    #TODO: to możnaby w sumie jakoś ładniej podzielić
    def run_model(self):
        #i tutaj jakoś sprawdzenie długości nagrania i ewentualne odpalenie kilka razy
        loadedAudio = AudioSegment.from_wav(self.path)

        time = loadedAudio.duration_seconds

        #dodanie ciszy, tak żeby długość nagrania była podzielna przez 10
        if time%10 !=0:
            silent_time = 10.00 - time%10
            silent_time = np.ceil(silent_time)
        else:
            silent_time = 0
        silent_segment = AudioSegment.silent(duration=silent_time*1000) 

        loadedAudio = loadedAudio + silent_segment
        counter = int(loadedAudio.duration_seconds/10)
        loadedAudio = loadedAudio[0:10*counter*1000]
        
        model_predictions = []
        for i in range(counter):
            cutAudio = loadedAudio[i*10*1000:10*(i+1)*1000]
            cutAudio.export('processedAudio.wav', format="wav")
            result = self.trainer.test(model=self.model,test_dataloaders = DataLoader(TestDataset( './processedAudio.wav', 1), batch_size = 64) )
            model_predictions.append( result[0]["test_epoch_end_accuracy"] )

        os.remove('./processedAudio.wav')
        if 1 in model_predictions:
            return 1
        else:
            return 0



In [5]:
print(len(files_list))

320


In [6]:
hasbird =0
for file in files_list:
    classifier = Classifier("./BirdVox/data/wav/"+file,file)
    print(classifier.model_prediction)
    if classifier.model_prediction==1:
        hasbird = hasbird+1

GPU available: True, used: False
TPU available: None, using: 0 TPU cores
/home/utworzyd/miniconda3/envs/pytorch_2/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: GPU available but not used. Set the --gpus flag when calling the script.
  warnings.warn(*args, **kwargs)
/home/utworzyd/miniconda3/envs/pytorch_2/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Testing: 0it [00:00, ?it/s]

/home/utworzyd/miniconda3/envs/pytorch_2/lib/python3.8/site-packages/torch/functional.py:516: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729062494/work/aten/src/ATen/native/SpectralOps.cpp:590.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore
/home/utworzyd/miniconda3/envs/pytorch_2/lib/python3.8/site-packages/pytorch_lightning/core/step_result.py:148: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  value = torch.tensor(value, device=device, dtype=torch.float)
GPU available: True, used: False
TPU available: None, using: 0 TPU cores


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_epoch_end_accuracy': 0.0, 'test_epoch_end_loss': 1.1573491096496582}
--------------------------------------------------------------------------------
0


Testing: 0it [00:00, ?it/s]

...

Testing: 0it [00:00, ?it/s]

GPU available: True, used: False
TPU available: None, using: 0 TPU cores


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_epoch_end_accuracy': 0.0, 'test_epoch_end_loss': 2.832016944885254}
--------------------------------------------------------------------------------
0


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_epoch_end_accuracy': 0.0, 'test_epoch_end_loss': 2.340453624725342}
--------------------------------------------------------------------------------
0


In [7]:
print(hasbird)

106
